In [101]:
import torch.nn as nn
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

class Model(nn.Module):
    def __init__(self) -> None:
        super(Model, self).__init__()
        self.stack = nn.Sequential(
            nn.Linear(5, 8),
            ##nn.Linear(8, 4),
            nn.Linear(8, 1)
        )
    def forward(self, x):
        return self.stack(x)


Using cpu device


In [100]:
from torch.utils.data import Dataset
import pandas as pd
from torch import Tensor

class MyDataset(Dataset):
    def __init__(self, df: pd.DataFrame) -> None:
        df = df.dropna(ignore_index = True)
        df.to_csv('outputdf.csv')
        self.input = Tensor(df[['val1', 'val2', 'val3', 'val4', 'val5']].astype('float32').values)
        self.target = Tensor(df['target'].astype('float32').values)
    def __len__(self):
        return len(self.input)
    def __getitem__(self, index):
        return self.input[index], self.target[index]
    
whole_dataset = MyDataset(pd.read_csv('spy.csv'))

In [42]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        
        pred = model(X)
        loss = loss_fn(pred.flatten(), y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch % 10 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f'loss: {loss:>7f} [{current:>5d}/{size:>5d}]')

In [114]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    money = 0
    bh_money = 0
    entries = 0
    bh_entries = 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)

            pred = model(X)
            for i in range(len(pred)):
                bh_money += y[i]
                if (pred[i][0] > 0):
                    money += y[i]
                    entries += 1
                bh_entries +=1
    print(f"Beat BH by {money-bh_money}, avg {money/entries} vs {bh_money/bh_entries}")

In [120]:
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
import os

model = Model().to(device)
if os.path.exists('model2.pth'):
    print("GRAB FROM FILE!!!")
    model.load_state_dict(torch.load('model2.pth'))
loss_fn = nn.MSELoss()
optim_fn = optim.SGD(model.parameters(), lr=0.00001)

train_count = int(len(whole_dataset) * 0.7)
test_count = len(whole_dataset) - train_count
train_dataset, test_dataset = random_split(whole_dataset, [train_count, test_count])

train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=True)

n_epochs = 2000000
for t in range(n_epochs):
    print(f'Epoch {t+1}')
    train(train_dataloader, model, loss_fn, optim_fn)
    test(train_dataloader, model, loss_fn)

Epoch 1
loss: 68.973961 [    0/  310]
loss: 72.361992 [  160/  310]
Beat BH by 12.17999267578125, avg 1.3057326078414917 vs 1.253806233406067
Epoch 2
loss: 80.496147 [    0/  310]
loss: 78.428017 [  160/  310]
Beat BH by 9.410003662109375, avg 1.2883169651031494 vs 1.253806233406067
Epoch 3
loss: 101.505470 [    0/  310]
loss: 66.780624 [  160/  310]
Beat BH by 0.0, avg 1.253806233406067 vs 1.253806233406067
Epoch 4
loss: 61.622868 [    0/  310]
loss: 70.879768 [  160/  310]
Beat BH by 0.0, avg 1.253806233406067 vs 1.253806233406067
Epoch 5
loss: 113.754425 [    0/  310]
loss: 69.166138 [  160/  310]
Beat BH by 0.0, avg 1.253806233406067 vs 1.253806233406067
Epoch 6
loss: 148.557556 [    0/  310]
loss: 151.332245 [  160/  310]
Beat BH by 0.0, avg 1.253806233406067 vs 1.253806233406067
Epoch 7
loss: 81.894310 [    0/  310]
loss: 70.288498 [  160/  310]
Beat BH by 0.0, avg 1.253806233406067 vs 1.253806233406067
Epoch 8
loss: 79.720482 [    0/  310]
loss: 54.305359 [  160/  310]
Beat BH b

KeyboardInterrupt: 

In [116]:
torch.save(model.state_dict(), "model2.pth")

In [110]:
predictor = Model().to(device)
predictor.load_state_dict(torch.load('model2.pth'))
print(predictor(Tensor([8.30439684,	10.69840144,	4.555859962,	0.462583402,	0.282509586
])))

tensor([-0.0455], grad_fn=<ViewBackward0>)
